<a href="https://colab.research.google.com/github/dewangmodi/Debiasing-KGE/blob/master/g_score(incomplete)%20and%20debias%20projection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
DIMS = 50
CUTOFF = 0.00002

In [ ]:
entity_vec, relations_vec = load_embeddings(dims=DIMS)
entity_dict, relations_dict = load_ids()

In [ ]:
male_vector, female_vector = get_gender_vectors(entity_vec, entity_dict, DIMS)
triplets, entity_labels, property_labels = load_data()
male_occupations, female_occupations, neutral_occupations = get_male_female_neutral_occupations(triplets, entity_labels, property_labels, CUTOFF)

In [ ]:
import math
def cosine(v1,v2):
  return np.dot(v1.transpose(),v2)/(np.linalg.norm(v1)*np.linalg.norm(v2))

In [ ]:
def get_g_score(occupation_list):
  gm_score,gf_score=0,0
  count=0
  for i in occupation_list:
        occupation_id = get_entity_id(entity_labels, i)
        occupation_id_with_vector = []
        for occ_id in occupation_id:
            if occ_id in entity_dict:
                occupation_id_with_vector.append(occ_id)
        if len(occupation_id_with_vector)==0:
            continue
        occupation_vector = get_averaged_entity_vector(entity_vec, entity_dict,DIMS,occupation_id_with_vector)
        g1_score=cosine(occupation_vector,male_vector/np.linalg.norm(male_vector))
        g2_score=cosine(occupation_vector,female_vector)
        gm_score+=(g1_score)
        gf_score+=(g2_score)
        count+=1
  return gm_score/count,gf_score/count,count
  

In [ ]:
m1,m2,c=get_g_score(female_occupations)
print(m1,m2,c)

[[0.32123734]] [[0.3276567]] 312


In [ ]:
def get_occupation_score_debias(occupation_list,lamb):
  gv1=(female_vector-male_vector)
  nm=np.linalg.norm(gv1)
  gender_vector = (female_vector - male_vector)/np.linalg.norm(female_vector - male_vector)
  projection_score = 0
  count = 0
  for i in occupation_list:
      occupation_id = get_entity_id(entity_labels, i)
      occupation_id_with_vector = []
      for occ_id in occupation_id:
          if occ_id in entity_dict:
              occupation_id_with_vector.append(occ_id)
      if len(occupation_id_with_vector)==0:
          continue
      occupation_vector = get_averaged_entity_vector(entity_vec, entity_dict,DIMS,occupation_id_with_vector)
      projection = np.dot(gender_vector.transpose(), occupation_vector)
      gk=projection
      occupation_vector1=occupation_vector-lamb*gk*gender_vector
      projection=np.dot(gender_vector.transpose(), occupation_vector1)
      projection_score += projection
      count += 1
  return projection_score/count


In [ ]:
male_occupation_score = get_occupation_score_debias(male_occupations,0.5)
female_occupation_score = get_occupation_score_debias(female_occupations,0.5)
neutral_occupation_score = get_occupation_score_debias(neutral_occupations,0.5)
print("Projection Score\tMale\t\tFemale\t\tNeutral")
print("\t\t\t%f\t%f\t%f"%(male_occupation_score, female_occupation_score, neutral_occupation_score))

Projection Score	Male		Female		Neutral
			0.088583	0.077916	0.052209
